Данные отсюда: https://nijianmo.github.io/amazon/index.html, категория All beauty (reviews и metadata)

Способы:
1. Написать правила с помощью natasha/yargy/spacy. Дополнительные данные не нужны. Плюсы: хороший результат, мало неоднозначности. Минусы: долго. 
2. Посмотреть metadata. Найти упоминания в столбце title, составить массив комментариев к этим товарам, найти ne в комментариях, например, при помощи словаря сущностей. Используются метаданные. Плюсы: легко, точно найдутся отзывы на нужный товар. Минусы: недостаточно просто найти отзывы на товар, нужно еще смотреть упоминания, так что получится довольно долго. 
3. Составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec). Плюсы: просто, быстро. Минусы: что-то можно не учесть.

Я буду использовать spacy (он подходит для английского).


In [ ]:
!pip install natasha

In [3]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('All_Beauty.json.gz')
meta = getDF('meta_All_Beauty.json.gz')

In [131]:
from collections import Counter

In [56]:
from sklearn.utils import shuffle
df = shuffle(df[['reviewText', 'asin' ]])
df

,reviewText,asin
275725,"For the color and consistency of this product,...",B005GMYQ1Q
286080,I love these. They are all small but otherwise...,B00CBP9U5I
304559,Perfect match to the shirt I also ordered!,B00KPHK3KA
234238,Did absolutely nothing to cure beardruff.,B01CDEJ1H8
323634,"The description is 5pair , I only received one...",B00UL6HUQA
...,...,...
318278,Warning the pump is horrible its by air thrust...,B00R24HVDQ
148451,Just as expected,B00F029PWC
341433,It gave me a rash all over my body by the thir...,B016V8YWBC
334656,the hole is a little too big,B013GP61BC


In [5]:
meta = shuffle(meta[['category','title','asin' ]])
meta

,category,title,asin
18836,[],Pokemon Happy Birthday Balloon Bouquet,B00SLYZF1G
12649,[],"Collection Lasting Perfection Concealer, Cool ...",B00IPG1ZQ4
21567,[],Nmark Muji Mild Scrub Face Wash Form 120g (Gre...,B00ZR3YNVM
18598,[],Disney Princess Lip Gloss Fruity Flavors 6 Pack,B00S1HC6WE
31455,[],No7 Beautiful Skin Dark Circle Corrector,B01FOREP0Y
...,...,...,...
15251,[],EX1 Cosmetics Invisiwear Liquid Foundation Num...,B00M681DWI
3321,[],Panasonic WES9839P Men's Electric Razor Replac...,B001J6O0NW
16179,[],Hello Breath Spray - Mojito Mint .24 oz. (Pack...,B00NB588BE
15993,[],"Kiss Double-Up TopGel Nail Top Coat, 60656, 0....",B00N3TUURI


In [6]:
full_data = df.merge(meta, on=["asin"])
full_data

,reviewText,asin,category,title
0,Love it!,B0067F28ZW,[],Eyelash Growth Serum - 100% Natural Eyelash Ey...
1,This brush set is good just a bit short ..but ...,B0067F28ZW,[],Eyelash Growth Serum - 100% Natural Eyelash Ey...
2,great brushes for a good value,B0067F28ZW,[],Eyelash Growth Serum - 100% Natural Eyelash Ey...
3,"i like these brushes, the eyebrow brush came w...",B0067F28ZW,[],Eyelash Growth Serum - 100% Natural Eyelash Ey...
4,Low quality didn't last,B0067F28ZW,[],Eyelash Growth Serum - 100% Natural Eyelash Ey...
...,...,...,...,...
382527,I really like these bifocal sunglasses. I hav...,B006XZG6BS,[],Town and Country Designer Bifocal Sunglasses f...
382528,Exelent product for pregnant womens and after ...,B0045PC92I,[],Calciodoron PM-50 g Brand: Weleda
382529,This is not a strong product for the clinic,B010PS0LEG,[],AHRONG ELTECH REX-KARA II Skin Massager Skin C...
382530,This gives you an amazing flawless finish. I h...,B01CDW6O6G,[],Tarte Rainforest of the Sea Water Foundation L...


In [58]:
data = full_data[:100000]

In [61]:
n = 0
for i in data['reviewText'][:300000]:
  if 'lipstick' or 'chapstick' in str(i):
    #print(i)
    n+=1
print(n)

100000


Будем искать помады.

In [ ]:
!pip install spacy

In [46]:
import spacy
from spacy.matcher import Matcher

In [40]:
from spacy.lang.en import English

In [41]:
nlp = English()
doc = nlp('This is not a strong product for the clinic')

In [49]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"POS": "ADJF"},
    {"LOWER": "product"}
]
matcher.add("fifa_pattern", None, pattern)
doc = nlp("This is not a strong product for the clinic")
matches = matcher(doc)
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span)

In [63]:
import re

In [124]:
pattern1 =  [
    {"POS": "ADJF"},
    {"LOWER": "lipstick"}
]
pattern2 =  [
    {"POS": "ADJF"},
    {"LOWER": "chapstick"}
]
pattern3 =  [
    {"LOWER": "this"},
    {"LOWER": "lipstick"}
]
pattern4 =  [
    {"LOWER": "this"},
    {"LOWER": "chapstick"}
]
pattern5 =  [
    {"LOWER": "lipstick"},
    {"POS": "NOUN"}
]
pattern6 =  [
    {"LOWER": "chapstick"},
    {"POS": "NOUN"}
]
pattern7 =  [
    {"LOWER": "chapstick|lipstick"},
    {"POS": "NOUN"}
]
pattern8 =  [
    {"POS": "ADJF"},
    {"LOWER": "product"}
]
pattern9 =  [
    {"POS": "ADJF"},
    {"TEXT": r"\w+"},
    {"LOWER": "chapstick|lipstick|thing"}
]

In [111]:
matcher.add("ne_patterns", None, pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7, pattern8, pattern9)

In [113]:
ne = []
#беру 50000, потому что на бОльшей выборке очень долго работает:(
for item in list(data['reviewText'][:50000]):
    doc = nlp(str(item))
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches] 
    ne.append(spans)

In [ ]:
ne = [str(element) for element in ne]
exclusive = list(Counter(ne).keys())
print(exclusive)

In [128]:
left_pattern1 =  [
    {"LENGTH": {"<=": 30}},
    {"POS": "ADJF"},
    {"LOWER": "lipstick"}
]
left_pattern2 =  [
    {"LENGTH": {"<=": 30}},
    {"POS": "ADJF"},
    {"LOWER": "chapstick"}
]
left_pattern3 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "this"},
    {"LOWER": "lipstick"}
]
left_pattern4 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "this"},
    {"LOWER": "chapstick"}
]
left_pattern5 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "lipstick"},
    {"POS": "NOUN"}
]
left_pattern6 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "chapstick"},
    {"POS": "NOUN"}
]
left_pattern7 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "chapstick|lipstick"},
    {"POS": "NOUN"}
]
left_pattern8 =  [
    {"LENGTH": {"<=": 30}},
    {"POS": "ADJF"},
    {"TEXT": r"\w+"},
    {"LOWER": "chapstick|lipstick|thing"}
]
left_pattern9 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "this"},
    {"TEXT": r"\w+"},
    {"LOWER": "chapstick|lipstick|thing"}
]

In [130]:
matcher = Matcher(nlp.vocab)
matcher.add("left_patterns", None, left_pattern1, left_pattern2, left_pattern3, left_pattern4, left_pattern5, left_pattern6, left_pattern7, left_pattern8, left_pattern9)

In [132]:
left = []
for item in list(data['reviewText'][:50000]):
    doc = nlp(str(item))
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches] 
    left.append(spans)

In [135]:
left = [str(element) for element in left]
left_ngramm = list(Counter(left).keys())
print(left_ngramm)

['[]', '[tried this lipstick]', '[and lipstick base]', '[with this lipstick]', '[of this lipstick]', '[and this lipstick]', '[love this lipstick]', '[describe this lipstick]', '[general lipstick bullet]', '[a lipstick crayon]', '[my lipstick collection]', '[each lipstick case]', '[these lipstick cases]', '[worn lipstick case]', '[the lipstick cases]', '[quality lipstick cases]', '[your lipstick case]', '[good lipstick holders]', '[Nice lipstick cases]', '[your lipstick colors]', '[casual lipstick wearer]', '[buy lipstick pencils]', '[many lipstick colors]', '[the lipstick tone, lighter lipstick color, same lipstick color]', '[different lipstick colors]', '[as lipstick colors]', '[new lipstick shades]', '[these lipstick liners]', '[my lipstick shades]', '[cheap lipstick smell]', '[your lipstick collection]', '[my lipstick color]', '[as lipstick substitute]', '[end lipstick liner]', '[Liquid Lipstick set]', '[a lipstick person]', '[used lipstick colors]', '[your lipstick fades]', '[a lip

In [145]:
right_pattern1 =  [
    {"POS": "ADJF"},
    {"LOWER": "lipstick"},
    {"LENGTH": {"<=": 30}}
]
right_pattern2 =  [
    {"POS": "ADJF"},
    {"LOWER": "chapstick"},
    {"LENGTH": {"<=": 30}}
]
right_pattern3 =  [
    {"LENGTH": {"<=": 30}},
    {"LOWER": "this"},
    {"LOWER": "lipstick"}
]
right_pattern4 =  [
    {"LOWER": "this"},
    {"LOWER": "chapstick"},
    {"LENGTH": {"<=": 30}}
]
right_pattern5 =  [
    {"LOWER": "lipstick"},
    {"POS": "NOUN"},
    {"LENGTH": {"<=": 30}}
]
right_pattern6 =  [
    {"LOWER": "chapstick"},
    {"POS": "NOUN"},
    {"LENGTH": {"<=": 30}}
]
right_pattern7 =  [
    {"LOWER": "chapstick|lipstick"},
    {"POS": "NOUN"}
    {"LENGTH": {"<=": 30}}
]
right_pattern8 =  [
    {"POS": "ADJF"},
    {"TEXT": r"\w+"},
    {"LOWER": "chapstick|lipstick|thing"},
    {"LENGTH": {"<=": 30}}
]
right_pattern9 =  [
    {"LOWER": "this"},
    {"TEXT": r"\w+"},
    {"LOWER": "chapstick|lipstick|thing"},
    {"LENGTH": {"<=": 30}}
]

In [146]:
matcher = Matcher(nlp.vocab)
matcher.add("right_patterns", None, right_pattern1, right_pattern2, right_pattern3, right_pattern4, right_pattern5, right_pattern6, right_pattern7, right_pattern8, right_pattern9)

In [147]:
right = []
for item in list(data['reviewText'][:50000]):
    doc = nlp(str(item))
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches] 
    right.append(spans)

In [ ]:
right = [str(element) for element in left]
right_ngramm = list(Counter(right).keys())
print(right_ngramm)